In [328]:
# Load data
with open("../data/day19.txt", "r") as f:
    input = f.read()

parts = input.split("\n\n")
rules = parts[0]
msgs = parts[1].split("\n")

rule_dict = {int(rule.split(":")[0]):rule.split(":")[1][1:] for rule in rules.split("\n")}
#rule_dict

# maybe parse to better data structure first

In [329]:
rule_dict[0]

'8 11'

In [330]:
rule_dict


{17: '72 112 | 71 3',
 31: '71 33 | 72 57',
 49: '72 110 | 71 45',
 37: '71 68 | 72 45',
 124: '71 128 | 72 64',
 1: '105 71 | 29 72',
 44: '72 130 | 71 93',
 93: '72 45 | 71 105',
 126: '34 71 | 59 72',
 89: '71 16 | 72 118',
 92: '21 71 | 12 72',
 55: '47 72 | 61 71',
 7: '29 72 | 29 71',
 45: '72 72 | 72 71',
 28: '110 71 | 109 72',
 10: '110 72 | 63 71',
 29: '72 71',
 27: '71 53 | 72 13',
 107: '45 71',
 84: '5 71 | 39 72',
 119: '72 105 | 71 110',
 53: '28 71 | 10 72',
 18: '72 86 | 71 71',
 25: '110 72 | 105 71',
 82: '106 71 | 75 72',
 46: '29 71 | 63 72',
 71: '"a"',
 129: '49 71 | 80 72',
 123: '12 71 | 79 72',
 111: '110 72 | 96 71',
 35: '72 107 | 71 104',
 113: '10 72 | 1 71',
 106: '26 72 | 6 71',
 8: '42',
 36: '71 10 | 72 79',
 26: '72 122 | 71 51',
 74: '48 72 | 2 71',
 5: '72 104 | 71 22',
 120: '71 48 | 72 2',
 64: '72 96 | 71 56',
 72: '"b"',
 131: '63 72 | 56 71',
 79: '56 71',
 23: '72 96 | 71 29',
 6: '71 111 | 72 32',
 43: '71 73 | 72 70',
 78: '14 71 | 74 72',


In [323]:
regex_rule = re.compile(r"(?P<TwoSequences>\d+ \d+ \| \d+ \d+)|(?P<Sequence>(?:\d+ ?)+)|\"(?P<Letter>[ab])\"")

for rule_to_parse in rule_dict.values():
    regex_results = regex_rule.search(rule_to_parse)
    print(rule_to_parse, regex_results)

4 1 5 6 <re.Match object; span=(0, 7), match='4 1 5 6'>
2 3 | 3 2 <re.Match object; span=(0, 9), match='2 3 | 3 2'>
4 4 | 5 5 <re.Match object; span=(0, 9), match='4 4 | 5 5'>
4 5 | 5 4 <re.Match object; span=(0, 9), match='4 5 | 5 4'>
"a" <re.Match object; span=(0, 3), match='"a"'>
"b" <re.Match object; span=(0, 3), match='"b"'>
4 5 <re.Match object; span=(0, 3), match='4 5'>


In [324]:
# Puzzle 1 with objects

# Not trees per se, but object-oriented, to easily copy objects.

# Parsing function:
# if a/b: store value, drop doing, move one of todo to doing, stop if todo is empty
# if 1 2 3: put first into doing, rest into todo. 
# if 1 2 | 2 1: copy object, put both firsts into doing, rest into todo. call parsing functions of both

# Quite slow, gives almost 1 million options. 62 is not correct!
# Try another test, modify day19test.txt

import re
regex_rule = re.compile(r"(?P<TwoSequences>\d+ \d+ \| \d+ \d+)|(?P<Sequence>(?:\d+ ?)+)|\"(?P<Letter>[ab])\"")

class RuleSet:
    """
    Main class containing all of the rules and their results.
    """
    def __init__(self, rule_dict):
        self.rule_dict = rule_dict
        self.rules = []

        self.parse()

    def parse(self):
        """Create initial rule, also refer back to RuleSet to be able to add new rules dynamically."""
        initial_rule = Rule(self.rule_dict, ruleset=self)
        self.rules.append(initial_rule)

    def get_results(self):
        """Return list of results."""
        return [rule.result for rule in self.rules]

class Rule():
    """
    Class to recursively parse the ruleset, creates new instance if a split is encountered.
    """
    def __init__(self, rule_dict, todo=[], result="", ruleset=None, start_n=0):
        self.rule_dict = rule_dict
        self.todo = todo
        self.result = result
        self.ruleset = ruleset

        self.parse(start_n)

    def parse(self, n=0):
        rule_to_parse = self.rule_dict[n]
        regex_results = regex_rule.search(rule_to_parse)

        if regex_results["TwoSequences"]:
            parts = regex_results["TwoSequences"].split("|")

            queue = self.sequence_to_queue(parts[1])
            todo = queue + self.todo
            rule_new = Rule(self.rule_dict, todo, self.result, self.ruleset, todo.pop(0))
            self.ruleset.rules.append(rule_new)

            self.parse_sequence(parts[0])
            
        if regex_results["Sequence"]:
            self.parse_sequence(regex_results["Sequence"])

        if regex_results["Letter"]:
            if regex_results["Letter"] == "a":
                self.result += "a"
            if regex_results["Letter"] == "b":
                self.result += "b"
            if self.todo:
                self.parse(self.todo.pop(0))
        return

    def sequence_to_queue(self, sequence: str):
        """Parse 1 2 3 to [1, 2, 3]"""
        return [int(value) for value in sequence.strip().split(" ")]

    def parse_sequence(self, sequence: str):
        """Parse 1 2 3"""
        queue = self.sequence_to_queue(sequence)
        self.todo = queue + self.todo
        self.parse(self.todo.pop(0))


rules = RuleSet(rule_dict)
allowed_messages = rules.get_results()

print(allowed_messages[0:5])

['ababbbab', 'abaaabab', 'aabbbbab', 'aabaabab', 'abbbabab']


In [325]:
len(allowed_messages)

8

In [326]:
n = 0
for msg in msgs:
    if msg in allowed_messages:
        n += 1

print(n)

2
